# **Project - Banking Transaction Management System Using Balanced Trees**



#### Importing necessary libraries:

In [1]:
import time as t
import random

### 🧾 Class `Transaction`
First, we must define the structure of a banking transaction, with the following fields:
- `id`: unique identifier generated randomly;
- `time`: insertion timestamp;
- `value`: transaction amount (positive for deposits, negative for withdrawals);
- `type`: transaction type, automatically defined based on the value.

In [7]:
class Transaction:
    def __init__(self, trans_id, time, value):
        self.id = trans_id
        self.time = time
        self.value = value
        self.type = "deposit" if value >= 0 else "withdrawal"

    def __str__(self):
        return f"Time: {self.time}, ID: {self.id}, Value: {self.value:.2f}, Type: {self.type}"

### 🌲 Class `AVLTree`
**Insertion**

Implements node insertion into the AVL tree. The height of each node is updated after every operation, and balance is maintained through single or double rotations whenever necessary.

**Removal, Search, and Utility Methods**
- `delete`: removes a node based on the `time` field and rebalances the tree.
- `search`: searches for a transaction with a specific timestamp.
- `inorder`: traverses the tree in ascending order of time.
- Auxiliary functions handle height calculation, balancing, and rotations.

In [2]:
# Class representing an AVL tree node
class AVLNode:
    def __init__(self, transaction):
        self.transaction = transaction  # Transaction object stored in the node
        self.left = None                # Left child
        self.right = None               # Right child
        self.height = 1                 # Node height (leaf starts with height 1)

# Class representing the AVL tree
class AVLTree:
    # Function to insert a new transaction into the tree
    def insert(self, root, transaction):
        # Base case: if the tree is empty, create a new node
        if not root:
            return AVLNode(transaction)

        # Recursive insertion to the left or right based on time (key)
        if transaction.time < root.transaction.time:
            root.left = self.insert(root.left, transaction)
        else:
            root.right = self.insert(root.right, transaction)

        # Update the height of the current node after insertion
        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        # Calculate the balance factor to check if the tree became unbalanced
        balance = self.get_balance(root)

        # ---- Unbalancing cases and correction via rotations ----

        # Case 1: Insertion in the left subtree of the left child (Left-Left)
        if balance > 1 and transaction.time < root.left.transaction.time:
            return self.right_rotate(root)

        # Case 2: Insertion in the right subtree of the right child (Right-Right)
        if balance < -1 and transaction.time > root.right.transaction.time:
            return self.left_rotate(root)

        # Case 3: Insertion in the right subtree of the left child (Left-Right)
        if balance > 1 and transaction.time > root.left.transaction.time:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        # Case 4: Insertion in the left subtree of the right child (Right-Left)
        if balance < -1 and transaction.time < root.right.transaction.time:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        # Return the (possibly rotated) node to the upper level of recursion
        return root

    def delete(self, root, time_key):
        # Base case: empty tree
        if not root:
            return root

        # Recursively traverse the tree to find the node to be removed
        if time_key < root.transaction.time:
            root.left = self.delete(root.left, time_key)
        elif time_key > root.transaction.time:
            root.right = self.delete(root.right, time_key)
        else:
            # Node found: 3 removal cases

            # Case 1: node with no left child
            if not root.left:
                return root.right

            # Case 2: node with no right child
            elif not root.right:
                return root.left

            # Case 3: node with two children
            # Replace the node with the smallest value in the right subtree
            temp = self.get_min_value_node(root.right)
            root.transaction = temp.transaction

            # Remove the successor node that was moved
            root.right = self.delete(root.right, temp.transaction.time)

        # Update the height of the current node
        root.height = 1 + max(self.get_height(root.left), self.get_height(root.right))

        # Calculate the balance factor to check if the node is unbalanced
        balance = self.get_balance(root)

        # Left-Left case unbalancing
        if balance > 1 and self.get_balance(root.left) >= 0:
            return self.right_rotate(root)

        # Left-Right case
        if balance > 1 and self.get_balance(root.left) < 0:
            root.left = self.left_rotate(root.left)
            return self.right_rotate(root)

        # Right-Right case
        if balance < -1 and self.get_balance(root.right) <= 0:
            return self.left_rotate(root)

        # Right-Left case
        if balance < -1 and self.get_balance(root.right) > 0:
            root.right = self.right_rotate(root.right)
            return self.left_rotate(root)

        return root

    def search(self, root, time_key):
        # Binary search by `time` field
        if not root:
            return None
        if root.transaction.time == time_key:
            return root.transaction
        elif time_key < root.transaction.time:
            return self.search(root.left, time_key)
        else:
            return self.search(root.right, time_key)

    def inorder(self, root):
        # In-order traversal (in ascending order of time)
        if root:
            self.inorder(root.left)
            print(root.transaction)
            self.inorder(root.right)

    def get_height(self, node):
        # Returns the height of the node (0 if null)
        return node.height if node else 0

    def get_balance(self, node):
        # Balance factor = left height - right height
        return self.get_height(node.left) - self.get_height(node.right) if node else 0

    def left_rotate(self, z):
        # Simple left rotation
        y = z.right
        T2 = y.left

        # Perform rotation
        y.left = z
        z.right = T2

        # Update heights
        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        return y

    def right_rotate(self, z):
        # Simple right rotation
        y = z.left
        T3 = y.right

        # Perform rotation
        y.right = z
        z.left = T3

        # Update heights
        z.height = 1 + max(self.get_height(z.left), self.get_height(z.right))
        y.height = 1 + max(self.get_height(y.left), self.get_height(y.right))

        return y

    def get_min_value_node(self, node):
        # Returns the leftmost node of the subtree (smallest value)
        while node.left:
            node = node.left
        return node


### 📥 Insertion of 5000 Transactions
Generates 5000 random transactions, with varying `id` and `value`, and inserts them into the AVL tree.  
Finally, it displays the height of the tree as a balance metric.

In [5]:
tree = AVLTree()
root = None
time_counter = 0

for _ in range(5000):
    trans_id = random.randint(1000, 999999)
    value = round(random.uniform(-1000, 1000), 2)
    root = tree.insert(root, Transaction(trans_id, time_counter, value))
    time_counter += 1

print(f"Tree height after insertions: {tree.get_height(root)}")

Tree height after insertions: 13


### 🔍 Search by Time
Searches for 10 specific timestamps in the AVL tree and measures the total time required for all searches.

In [8]:
search_keys = [100, 1000, 1500, 2000, 2500, 3000, 3500, 4000, 4500, 4999]
start_time = t.time()
for key in search_keys:
    result = tree.search(root, key)
    if result:
        print(result)
    else:
        print(f"Transaction with time = {key} not found.")
end_time = t.time()
print(f"Total search time: {end_time - start_time:.6f} seconds")

Transaction with time = 100 not found.
Time: 1000, ID: 818946, Value: 717.15, Type: deposit
Time: 1500, ID: 917160, Value: -501.89, Type: withdrawal
Time: 2000, ID: 691740, Value: 798.39, Type: deposit
Time: 2500, ID: 883233, Value: 135.04, Type: deposit
Time: 3000, ID: 513177, Value: -372.14, Type: withdrawal
Time: 3500, ID: 557656, Value: 691.09, Type: deposit
Time: 4000, ID: 829744, Value: -511.50, Type: withdrawal
Time: 4500, ID: 351338, Value: 124.05, Type: deposit
Time: 4999, ID: 627086, Value: 319.59, Type: deposit
Total search time: 0.000420 seconds


### 🗑️ Removal of Old Transactions
Removes transactions with timestamps between 0 and 500, displays the tree height after deletion, and measures the total time spent.

In [6]:
start_time = t.time()
for key in range(0, 501):
    root = tree.delete(root, key)
end_time = t.time()
print(f"Height after removals: {tree.get_height(root)}")
print(f"Total removal time: {end_time - start_time:.6f} seconds")

Height after removals: 13
Total removal time: 0.003621 seconds


### 📤 Ordered Printing (In-order)
Traverses and prints all remaining transactions in chronological order using in-order traversal.

In [10]:
print("Remaining transactions (chronological order):")
tree.inorder(root)

Remaining transactions (chronological order):
Time: 501, ID: 313289, Value: 713.58, Type: deposit
Time: 502, ID: 541476, Value: -788.81, Type: withdrawal
Time: 503, ID: 582305, Value: 249.38, Type: deposit
Time: 504, ID: 170759, Value: -637.91, Type: withdrawal
Time: 505, ID: 876410, Value: -426.34, Type: withdrawal
Time: 506, ID: 218295, Value: -553.23, Type: withdrawal
Time: 507, ID: 329086, Value: 60.15, Type: deposit
Time: 508, ID: 259256, Value: -407.23, Type: withdrawal
Time: 509, ID: 322897, Value: -868.15, Type: withdrawal
Time: 510, ID: 89348, Value: 759.59, Type: deposit
Time: 511, ID: 770133, Value: -231.57, Type: withdrawal
Time: 512, ID: 343470, Value: -95.27, Type: withdrawal
Time: 513, ID: 341580, Value: 104.39, Type: deposit
Time: 514, ID: 56547, Value: -581.89, Type: withdrawal
Time: 515, ID: 86750, Value: 818.03, Type: deposit
Time: 516, ID: 381428, Value: 197.28, Type: deposit
Time: 517, ID: 191103, Value: 749.64, Type: deposit
Time: 518, ID: 558539, Value: 874.95, T

## Performance and Complexity Analysis of the AVL Tree

In this project, we implemented an AVL Tree to manage 5000 banking transactions, ensuring efficient operations even under heavy data load.

### Operations Performed
- Insertion of **5000 transactions**
- Search for **10 specific timestamps**
- Removal of **501 old transactions**
- Ordered printing of the remaining transactions (chronological order)

---

### Computational Complexity

The AVL Tree is a balanced data structure that maintains its height close to \(\log_2(n)\), ensuring that the main operations have logarithmic complexity:

| Operation | Complexity | Justification |
|------------|-------------|---------------|
| Insertion | O(log n) | After each insertion, at most one single or double rotation rebalances the tree. |
| Deletion | O(log n) | Similar to insertion, with possible rotations to maintain balance. |
| Search | O(log n) | Each comparison eliminates half of the remaining subtree. |
| Traversal | O(n) | In-order traversal visits all nodes to print them in order. |

---

### Theoretical vs. Practical Comparison

With \(n = 5000\) inserted transactions:

- **Theoretical maximum AVL height:**  
  \[
  h_{theoretical} \approx 1.44 \cdot \log_2(n) \approx 1.44 \cdot \log_2(5000) \approx 17.7
  \]

- **Observed (practical) height:**  
  \[
  h_{practical} = 13
  \]

This result shows that the tree remained **well-balanced** and below the expected theoretical limit, confirming the algorithm’s efficiency in a real-use scenario.

---

### Experimental Observations

- Searches for the 10 keys (timestamps) were quick and efficient.
- The removal of 501 transactions (from time 0 to 500) maintained structural integrity.
- The in-order traversal confirmed the correct chronological ordering of remaining transactions.
- The height of 13 demonstrates that even with 5000 elements, the tree’s depth remained low enough to ensure consistent performance.

---

### Conclusion

The AVL structure proved suitable for systems requiring high performance with dynamic and ordered data, such as a banking system.  
The **low practical tree height (13)** reinforces the AVL tree as an effective data structure, blending favorable theoretical complexity with excellent practical behavior.